In [ ]:
# %matplotlib qt

import numpy as np
import pandas as pd
import time 
import datetime 
import os

from src.dataset.parse_dataset import get_pIDs, prepare_data, get_profiles
from libs.visualisation import get_daily_glycaemic_variation, get_group_daily_glycaemic_variation, get_individual_plot, compare_measures, compare_glycaemic_measures

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [ ]:
dataset = 'OhioT1DM'  # 0:'Replace_BG, 1:'OhioT1DM, 2: 'OpenAPS', 3:'Tidepool'
dataset_path = './datasets/{}/raw/'.format(dataset)
pIDs = get_pIDs(dataset_path)
print(len(pIDs))
print(pIDs)

In [ ]:
if os.path.exists('./datasets/{}.csv'.format(dataset)):
    df = pd.read_csv('./datasets/{}.csv'.format(dataset))
else:
    df = prepare_data(dataset_path, pIDs, True)
    df.to_csv('./datasets/{}.csv'.format(dataset), index=False)

In [ ]:
if os.path.exists('./datasets/{}_profile.csv'.format(dataset)):
    profiles = pd.read_csv('./datasets/{}_profile.csv'.format(dataset))
else:
    profiles = get_profiles(dataset_path)
    profiles.to_csv('./datasets/{}_profile.csv'.format(dataset), index=False)

In [ ]:
pID = 540

In [ ]:
start_day = 10
end_day   = 12

display_plot = get_individual_plot(df, dataset, pID, start_day, end_day)
plt.show()

In [ ]:
display_plot = get_daily_glycaemic_variation(df, dataset, pID)
plt.show()

In [ ]:
category = "Gender"

display_plot = get_group_daily_glycaemic_variation(df, profiles, category)
plt.show()

In [ ]:
category = "Gender"
cat_array = profiles[category].unique()

fig, axs = plt.subplots(ncols= len(cat_array), figsize = (len(cat_array) * 5,6))
for ax, cat in zip(axs, cat_array):
    mask = profiles[category] == cat
    inc = profiles.loc[mask, 'pID']
    select_pIDs = inc.unique()
    ax, zone, zone_index = compare_measures(df, select_pIDs, ax)
    ax.set_title('[{cat}] Safe Regions: {per:.1f}%'.format(cat = cat, per = zone[0] + zone[1]))
# Ensure the layout is tight to prevent labels from being cut off
fig.tight_layout()
fig.suptitle(f'Clarke Error Grid stratified by {category}')
plt.show()

In [ ]:
measure = 'ADDR'
category = 'Age Range'
hue     = 'Gender'

sns_plot = compare_glycaemic_measures(df, profiles, pIDs, measure, category, hue, 0, 28)


In [ ]:
category = "Age Range"
cat_array = profiles.copy()[category].unique()
cat_array.sort()
df_group = df.copy()
for cat in cat_array:
    mask = profiles[category] == cat
    inc  = profiles.loc[mask, 'pID']
    select_pIDs = inc.unique()

    df_group.loc[df_group['pID'].isin(select_pIDs), category] = cat

sns.histplot(data=df_group, x='CGM' ,bins=30, stat = 'density', hue=category, kde=True, legend=True, common_norm=False)